In [1]:
%load_ext autoreload
%autoreload 2
# imports

# important libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as stats
import scipy as sp

from tqdm.auto import tqdm
import itertools

# for better printing of variables
from IPython.display import display

# custom imports
from concperf import single_model, general_model
from concperf import utility

In [5]:
# update configuration dictionary for each instance count
def update_config(config):
    config['arrival_rate_server'] = config['arrival_rate_total'] / config['instance_count']
    config['base_service_time'] = config['base_service_time_ms'] / 1000
    config['stable_conc_avg_count'] = 10 * config['instance_count']

model_config = {
    # 'instance_count' should be added for each state
    'max_conc': 100,
    'arrival_rate_total': 10,
    'alpha': 0.11,
    'base_service_time_ms': 1154,
    'max_container_count': 25,
    'target_conc': 1*0.7, # assumes target utilization
    'max_scale_up_rate': 1000, # from N to 1000*N at most
    'max_scale_down_rate': 2, # from N to N/2 at most
    # 'stable_conc_avg_count': 60, # number of times monitored concurrency will be averaged in stable mode
    'autoscaling_interval': 2, # amount of time between autoscaling evaluations
    'provision_rate_base': 1,
    'deprovision_rate_base': 2,
}

# test
# model_config = {
#     # 'instance_count' should be added for each state
#     'max_conc': 100,
#     'arrival_rate_total': 15,
#     'alpha': 1,
#     'base_service_time_ms': 400,
#     'max_container_count': 25,
#     'target_conc': 16*0.7, # assumes target utilization
#     'max_scale_up_rate': 1000, # from N to 1000*N at most
#     'max_scale_down_rate': 2, # from N to N/2 at most
#     # 'stable_conc_avg_count': 60, # number of times monitored concurrency will be averaged in stable mode
#     'autoscaling_interval': 2, # amount of time between autoscaling evaluations
#     'provision_rate_base': 1,
#     'deprovision_rate_base': 2,
# }

In [7]:
general_model.solve_general_model(model_config, update_config)

100%|██████████| 26/26 [00:45<00:00,  1.74s/it]


{'inst_count_probs': array([2.63907563e-07, 2.63845954e-07, 2.63820936e-07, 2.63824547e-07,
        2.63824174e-07, 2.63825883e-07, 2.63823215e-07, 2.63822875e-07,
        2.63819168e-07, 2.63819269e-07, 2.63815485e-07, 2.63810515e-07,
        2.63809917e-07, 2.63806094e-07, 2.63802879e-07, 2.63799653e-07,
        2.63799422e-07, 2.63803499e-07, 2.63803145e-07, 2.63800319e-07,
        2.63797254e-07, 2.63794919e-07, 2.63793021e-07, 2.63790285e-07,
        2.63788461e-07, 2.63786223e-07, 6.16836493e-07, 8.74163923e-07,
        5.68910680e-07, 5.68912128e-07, 5.68912301e-07, 5.68915528e-07,
        5.68923440e-07, 5.69007938e-07, 5.69521220e-07, 5.72008452e-07,
        5.81543851e-07, 6.10693902e-07, 6.81175439e-07, 8.13790261e-07,
        1.00190635e-06, 1.18928096e-06, 1.29259020e-06, 1.26757288e-06,
        1.14715774e-06, 1.00514105e-06, 8.93566639e-07, 8.09038001e-07,
        7.06849558e-07, 5.59679599e-07, 4.00706931e-07, 2.95658840e-07,
        2.63835685e-07, 2.63784939e-07, 2.63